In [0]:
pip install pymorphy2

     |████████████████████████████████| 51kB 2.6MB/s 
     |████████████████████████████████| 7.1MB 9.0MB/s 


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import re
import nltk
nltk.download('punkt')
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

path = '/content/drive/My Drive/Colab Notebooks/диплом/'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Открываем нужные файлы

In [0]:
#1. список Серенсена
with open(path + '9460-osnov-sanskritskikh-slov.txt', 'r', encoding = 'utf-8') as r:
    lines = r.readlines()
sans = [x.lower() for x in lines]
sans = [re.sub('\n', '', x) for x in sans]
print('Объем списка Серенсена - ' + str(len(sans)) + ' слов')


#2. список санскритизмов из 354 тыс. слов
with open(path + '384000.txt', 'r', encoding = 'utf-8') as n:
    sans_dict = n.readlines()
sans_dict = [re.sub('\n','',x) for x in sans_dict]
sans_dict = set(sans_dict)
###удаляем санскритизмы длиной 1-2 буквы
sans_dict = [x for x in sans_dict if len(x) > 2]

#3. словарь иностранных слов
with open(path + 'foreign_words.txt', 'r', encoding = 'utf-8') as n:
    d = n.readlines()
forn = [re.sub('\n', '', x) for x in d]
forn = [x for x in forn if len(x) > 1]
###добавляем к списку иностранных слов свои исключения, чтобы удалить их из корпуса русских слов (чтобы корпус русских слов не отфильтровывал нужные санскритизмы, например, преты)
for i in ['анил', 'анупа', 'ануп', 'крошу', 'ванг', 'прет']:
  forn.append(i)
print('Объем словаря иностранных слов - ' + str(len(forn)) + ' слов')

#4. корпус русских слов
with open(path + 'dict.opcorpora.txt', 'r', encoding = 'utf-8') as n:
    rus_words = []
    lem_rus_words = []
    d = n.read()
    d = re.sub('\n', ' ', d)
    d = re.sub('\t', ' ', d)
    d = re.sub('(^|\s)[0-9]+', '||', d)
    parads = re.findall('\|\|[^\|]+', d)
    for par in parads:
      try:
        lem = re.search('\|\|\s([А-Яа-яёЁ]+)', par).group(1)
        if 'Ё' in lem:
            lem = re.sub('Ё', 'Е', lem)
            
        if lem.lower() not in forn + sans:
            lem_rus_words.append(lem.lower())
            words = re.findall('[А-Яа-яЁё]+', par)
            for w in words:
                if 'Ё' in w:
                    w = re.sub('Ё', 'Е', w)
                rus_words.append(w.lower())
      except:
        continue
rus_words.remove('даму')
rus_words.remove('дама')
rus_words.remove('кишку')
rus_words.remove('пилу')
rus_words.remove('руру')
rus_words.remove('турья')
rus_words.remove('турье')
rus_words.remove('кшатрия')
rus_words.remove('кшатрии')
rus_words.append('и')
cases = ['nomn', 'gent', 'datv', 'accs', 'ablt', 'loct']
for case in cases:
  list(rus_words).append(morph.parse('акт')[0].inflect({case}).word)
rus_words = [x for x in rus_words if len(x) > 1]
rus_words = set(rus_words)
lem_rus_words = set(lem_rus_words)
print('Объем корпуса русских слов - ' + str(len(rus_words)) + ' слов')

###очищаем список санскритских слов от русских лемм
sans_dict2 = [z for z in sans_dict if z not in lem_rus_words]
print('Объем списка санскритских слов - ' + str(len(sans_dict2)) + ' слов')

Объем списка Серенсена - 9460 слов
Объем словаря иностранных слов - 14687 слов
Объем корпуса русских слов - 2914651 слов
Объем списка санскритских слов - 352592 слов


In [0]:
phrase = {}
repl = {}
index3 = []
opts = {}

#5. список рубрик с аннтоациями и пояснениями, отобранные из бумажного указателя к 3-му тому Махабхараты вручную
with open(path + '3_INDEX_phrases.txt', 'r', encoding='utf-8') as r:
  for i in r:
    s = i.split(': ')
    ph = s[0]
    syn = re.sub('\n', '', s[1])
    if ' ' in ph or '-' in ph:
      phrase[ph] =  syn
    else:
      repl[ph] =  syn

#6. список однословных рубрик из бумажного указателя
with open(path + '3_INDEX_oneword.txt', 'r', encoding='utf-8') as r:
  ones = r.readlines()
ones = [re.sub('\n', '', x) for x in ones]

#7. список рубрик с эпитетами
with open(path + '3_INDEX_options.txt', 'r', encoding = 'utf-8') as r:
  for i in r:
    m = i.split(': ')
    s = m[0].lower()
    index3.append(s)
    val = m[1].split('; ')
    arr = []
    for v in val:
      r = v.split(' - ')
      l = re.sub('\n', '', r[1])
      arr.append((r[0].lower(), l.lower()))
    opts[s] = arr

index3 += list(repl.keys()) + ones

In [0]:
#8. рубрики с русскими словами из бумажного указателя

with open(path + 'rus_index.txt', 'r', encoding='utf-8') as n:
  rus_index = n.readlines()
rus_index = [re.sub('\n', '', x) for x in rus_index]

#9. санскритизмы в плюралисе

with open(path + 'pluralis_племена.txt', 'r', encoding = 'utf-8') as r:
  tr = r.readlines()

tr = [re.sub('\n', '', x.lower()) for x in tr]

10.

with open(path + 'rusforms.txt', 'r', encoding = 'utf-8') as t:
    rusforms = t.readlines()
rusforms = [re.sub('\n', '', x) for x in rusforms]

Склонение русских слов из указателя

In [0]:
rus_index_one = []
for i in rus_index:
  if re.search('^[А-Яа-яёЁ]+$', i):
    rus_index_one.append(re.search('^[А-Яа-яёЁ]+$', i).group(0))
rus_index_phrase = list(set(rus_index) - set(rus_index_one))

rus_index = {}
cases = ['nomn', 'gent', 'datv', 'accs', 'ablt', 'loct']
for i in rus_index_one:
  for case in cases:
    if i not in rus_index:
      rus_index[i] = [morph.parse(i)[0].inflect({case}).word]
    else:
      rus_index[i].append(morph.parse(i)[0].inflect({case}).word)

both = ['вселенский владыка', 'великая смоковница', 'тройственная вселенная', 'мудрецы-цари', 'семь святых мудрецов', 'поминальная жертва', 'великий полководец', 'великий индра', 'святые мудрецы', 'мудрецы-боги', 'божественные мудрецы', 'великий бог', 'восемь чаш', 'тридцать богов', 'демоны-змеи', 'мудрецы-брахманы', 'брахманы-мудрецы', 'третье небо', 'три мира', 'долг, польза, любовь', 'тридцать три бога', 'великий владыка', 'цари-мудрецы', 'пять великих элементов']
last = ['не из чрева рожденный']
except_last = ['старший брат гады', 'веда гандхарвов']
tcsh = ['шествующий по небу', 'равно владеющий и правой, и левой рукой', 'смущающий душу', 'имеющий знаком быка', 'несущий знак']

for i in rus_index_phrase:
  st = i.split()
  if '-' in i:
    st = i.split('-')
  if i in both:
    arr = []
    for case in cases:
      for s in st:
        try:
          arr.append(morph.parse(s)[0].inflect({case}).word)
        except:
          continue
    arr = [(arr[indx], arr[indx+1]) for indx, x in enumerate(arr) if indx%2 == 0]
    rus_index[i] = [' '.join(x) for x in arr]
  elif i in tcsh:
    arr = []
    for fl in ['ему', 'ий', 'его', 'им', 'ем']:
      arr.append(re.sub('ий', fl, i))
    rus_index[i] = [''.join(x) for x in arr]
  elif i in except_last:
    arr = []
    for case in cases:
      for s in st:
        if s == st[-1]:
          arr.append(s)
          continue
        arr.append(morph.parse(s)[0].inflect({case}).word)
    if len(st) == 3:
      arr = [(arr[indx], arr[indx+1], arr[indx+2]) for indx, x in enumerate(arr) if indx%3 == 0]
    elif len(st) == 2:
      arr = [(arr[indx], arr[indx+1]) for indx, x in enumerate(arr) if indx%2 == 0]
    rus_index[i] = [' '.join(x) for x in arr]
  elif i in last:
      arr = []
      for case in cases:
        try:
          arr.append((st[0], st[1], st[2], morph.parse(st[3])[0].inflect({case}).word))
        except:
          continue
      rus_index[i] = [' '.join(x) for x in arr]
  else:
      arr = []
      for case in cases:
        try:
          arr.append((morph.parse(st[0])[0].inflect({case}).word, st[1]))
        except:
          continue
      rus_index[i] = [' '.join(x) for x in arr]

rus_index['сыновья дити'].append('сыновей дити')
rus_index['сыновья калаки'].append('сыновей калаки')

Разделение санскритизмов на 3 группы: несклоняемые; оканч. на гласную; окан. на согласную

In [0]:
indecl = []
decl = []
stem = []
vows = ['а', 'я', 'ы']
for i in index3 + sans + sans_dict2:
    if i.endswith('и') or i.endswith('у') or i.endswith('ю') or i.endswith('е') or i.endswith('о'):
        indecl.append(i)
    elif i[-1:] in vows:
        decl.append((i[:-1], i))
    else:
        stem.append(i)
for i in tr:
  decl.append((i[:-1], i))

indecl = set(indecl)
stem = set(stem)

In [0]:
len(set(decl))

243284

In [0]:
#exep - список исключений

preps = ['в', 'без', 'до', 'из', 'к', 'на', 'по', 'о', 'от', 'перед', 'при', 'через', 'с', 'у', 'за', 'над', 'об', 'под', 'про', 'для', 'ко', 'обо', 'ото', 'во', 'безо', 'передо', 'со', 'надо']
prons = ['меня', 'нем', 'наш', 'ваш', 'она', 'они', 'кто', 'что', 'ними', 'ней', 'наши', 'ваши', 'вашем', 'тот', 'та', 'те', 'той', 'нас', 'ней', 'нам', 'них']
conjs = ['или', 'но', 'да', 'либо', 'ради', 'нет', 'да']
advs = ['так', 'туда', 'как', 'тут', 'там', 'сами', 'сам', 'сама']
exep = preps + prons + conjs + advs

indecl = [x for x in indecl if x not in exep]

In [0]:
#словари с начальными буквами для ускоренной обработки
decl_d = {}
stem_d = {}

for i in list(set([x[0][:2] for x in decl])):
  decl_d[i] = []
for i in list(set([x[:2] for x in stem])):
  stem_d[i] = []

for i in decl:
  decl_d[i[0][:2]].append(i)
for i in stem:
  stem_d[i[:2]].append(i)

In [0]:
#список рубрик с дефисами
hyph = []

for i in list(phrase.keys()):
  if re.search('^[а-яА-ЯёЁ]+-[а-яА-ЯёЁ]+$', i):
    hyph.append(re.search('^[а-яА-ЯёЁ]+-[а-яА-ЯёЁ]+$', i).group(0))

rest = [x for x in list(phrase.keys()) if x not in list(rus_index.keys())]
hyph.append('тапас-брихадуктха')
phrase['тапас-брихадуктха'] = 'тапас-брихадуктха'

Поиск санскритизмов в русском тексте

In [0]:
def search(file):

  with open(path + file, 'r', encoding = 'utf-8') as r:
    text = r.read()

  reg_space = re.compile('\s{2,}', re.DOTALL)
  reg_punct = ("\s[0-9!\(\),-:;\?\[\]«»'–“„\.•…]+|[0-9!\(\),-:;\?\[\]«»'–“„\.•…]+\s|[0-9!\(\),-:;\?\[\]«»'–“„\.•…]+$|[\(\)«»'“„]+")
  sents = nltk.sent_tokenize(text)

  found = []
  upper_words = []

  count = 0
  for sent in sents:
    count += 1
    print('Обработано ' + str(count) + ' предложений из ' + str(len(sents)))

    if re.search('[а-яА-Я](-\s-[0-9]+-\s)[а-яА-Я]+', sent):
      br = re.search('[а-яА-Я](-\s-[0-9]+-\s)[а-яА-Я]+', sent).group(1)
      sent = re.sub(br, '', sent)

    text = re.sub(r'\d', '', sent)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'[A-Za-z]', '', text)
    text = re.sub(reg_punct, ' ', text)
  
    if re.search(reg_space, text):
      text = re.sub(reg_space, ' ', text)

    for k, vs in rus_index.items():
      if k in sent.lower() or k in text.lower():
        if k in phrase:
          found.append(phrase[k])
        else:
          found.append(k)
      else:
        for v in vs:
          if v in sent.lower() or v in text.lower():
            if k in phrase:
              found.append(phrase[k])
            else:
              found.append(k)

    for i in rest:
      if i in sent.lower() or i in text.lower():
        found.append(phrase[i])

    for i in hyph:
      arr = i.split('-')
      reg = '{}[а-я]+-{}[а-я]+'.format(arr[0][:-2], arr[1][:-2])
      if re.search(reg, sent.lower()):
        found.append(phrase[i])
      elif re.search(reg, text.lower()):
        found.append(phrase[i])

    vs = ['а', 'я', 'ю', 'е', 'о', 'ы', 'и', 'у']
    #проверяем каждое слово из текста
    for indx, word in enumerate(text.split()):
        f = False
        if word[0].isupper() == True and indx > 0:
          upper_words.append((word.lower(), indx, text))
        word = word.lower()
        #сохраняем слово, если оно в списке несклоняемых имен
        if word in indecl and word not in rus_words:
            found.append((word, indx, text))
          
        #список склоняемых санскритизмов, заканчивающихся на гласную
        if word[:2] in decl_d:
          if word in [x[1] for x in decl_d[word[:2]]]:
              found.append((word, indx, text))
              f = True
          else:
            for i in decl_d[word[:2]]:
                s = i[0]
                if word.startswith(s) and word[-1] in vs and len(word) - len(s) == 1 and word not in exep and word not in rus_words:
                    found.append((i[1], indx, text))
                    f = True
                elif len(word) > 2 and word.startswith(s) and word[-3] in vs and word.endswith('ми') and len(word) - len(s) == 3 and word not in exep and word not in rus_words:
                    found.append((i[1], indx, text))
                    f = True
                elif len(word) > 1 and word.startswith(s) and word[-2] in vs and word.endswith('м') and len(word) - len(s) == 2 and word not in exep and word not in rus_words:
                    if word[-2:] == 'ам' or word[-2:] == 'ям':
                      if i[1][-1] == 'и' or i[1][-1] == 'ы':
                        found.append((i[1], indx, text))
                        f = True
                    else:
                      found.append((i[1], indx, text))
                      f = True
                elif len(word) > 1 and word.startswith(s) and word[-2] in vs and word.endswith('в') and len(word) - len(s) == 2 and word not in exep and word not in rus_words:
                    if i[1][-1] == 'и' or i[1][-1] == 'ы':
                      found.append((i[1], indx, text))
                      f = True
                elif len(word) > 1 and word.startswith(s) and word[-2] in vs and word.endswith('й') and len(word) - len(s) == 2 and word not in exep and word not in rus_words:
                    found.append((i[1], indx, text))
                    f = True
                elif len(word) > 1 and word.startswith(s) and word[-2] in vs and word.endswith('х') and len(word) - len(s) == 2 and word not in exep and word not in rus_words:
                    if i[1][-1] == 'и' or i[1][-1] == 'ы':
                      found.append((i[1], indx, text))
                      f = True
                    elif word[:-1] == i[1]:
                      found.append((i[1], indx, text))
                      f = True


        if word[:2] in stem_d:
          if word in [x for x in stem_d[word[:2]]]:
            found.append((word, indx, text))
          else:
            for s in stem_d[word[:2]]:
                if word.startswith(s) and word[-1] != 'ь' and len(word) - len(s) <= 1 and word not in exep and word not in rus_words:
                  if word[-1] in vs and s[-1] in vs:
                    found.append((s, indx, text))
                  elif word[-1] not in vs and s[-1] not in vs:
                    found.append((s, indx, text))
                elif len(word) > 2 and word.startswith(s) and word[-3] in vs and word.endswith('ми') and len(word) - len(s) == 3 and word not in exep and word not in rus_words:
                    found.append((s, indx, text))
                elif len(word) > 1 and word.startswith(s) and word[-2] in vs and word.endswith('м') and len(word) - len(s) == 2 and word not in exep and word not in rus_words:
                  if word[-2:] == 'ам' or word[-2:] == 'ям':
                    if s[-1] == 'и' or s[-1] == 'ы':
                      found.append((s, indx, text))
                  else:
                    found.append((s, indx, text))
                elif len(word) > 1 and word.startswith(s) and word[-2] in vs and word.endswith('в') and len(word) - len(s) == 2 and word not in exep and word not in rus_words:
                    found.append((s, indx, text))
                elif len(word) > 1 and word.startswith(s) and word[-2] in vs and word.endswith('й') and len(word) - len(s) == 2 and word not in exep and word not in rus_words:
                    found.append((s, indx, text))
                elif len(word) > 1 and word.startswith(s) and word[-2] in vs and word.endswith('х') and len(word) - len(s) == 2 and word not in exep and word not in rus_words:
                    found.append((s, indx, text))

  return found, upper_words

In [0]:
#в скобках ввести название файла с текстом для обработка

found, upper_words = search('volume_3.txt')

Streaming output truncated to the last 5000 lines.
Обработано 12089 предложений из 17087
Обработано 12090 предложений из 17087
Обработано 12091 предложений из 17087
Обработано 12092 предложений из 17087
Обработано 12093 предложений из 17087
Обработано 12094 предложений из 17087
Обработано 12095 предложений из 17087
Обработано 12096 предложений из 17087
Обработано 12097 предложений из 17087
Обработано 12098 предложений из 17087
Обработано 12099 предложений из 17087
Обработано 12100 предложений из 17087
Обработано 12101 предложений из 17087
Обработано 12102 предложений из 17087
Обработано 12103 предложений из 17087
Обработано 12104 предложений из 17087
Обработано 12105 предложений из 17087
Обработано 12106 предложений из 17087
Обработано 12107 предложений из 17087
Обработано 12108 предложений из 17087
Обработано 12109 предложений из 17087
Обработано 12110 предложений из 17087
Обработано 12111 предложений из 17087
Обработано 12112 предложений из 17087
Обработано 12113 предложений из 17087

In [0]:
found = [x for x in found if x[0].endswith('ам') == False]

#добавляю невстречающиеся в тексте слова
to_append = {}
with open(path + 'append if found.txt', 'r', encoding='utf-8') as r:
  for i in r:
    s = i.split(': ')
    to_append[s[0]] = re.sub('\n', '', s[1])

for i in found:
  if isinstance(i, str) == True:
    if i in to_append:
      found.append(to_append[i])
  else:
    if i[0] in to_append:
      found.append(to_append[i[0]])


#переименовываю на эпитеты
ffound = []
for line in found:
  if isinstance(line, tuple):
    if line[0] not in opts:
      ffound.append(line)
    else:
      opt = opts[line[0]]
      f = False
      for i in opt:
        sp = i[0].split()
        for s in sp:
          if s[:-1] in line[2].lower():
            ffound.append((i[1], line[1], line[2]))
            f = True
      if f == False:
        ffound.append((line[0], line[1], line[2]))
  else:
    ffound.append([line])

sans = [x for x in sans if len(x) > 1]

Обработка слов с заглавной буквы

In [0]:
def capital_search(upper_words):
  found = []
  vs = ['а', 'я', 'ю', 'е', 'о', 'ы', 'и', 'у']
  for i in upper_words:
    word = i[0]
    for s in sans + index3 + tr:
        if word.startswith(s) and word[-1] != 'ь' and len(word) - len(s) <= 1 and word not in exep:
            found.append((s, i[1], i[2]))
        elif len(word) > 2 and word.startswith(s) and word[-3] in vs and word.endswith('ми') and len(word) - len(s) == 3 and word not in exep:
            #if s[-1] == 'и' or s[-1] == 'ы':
            found.append((s, i[1], i[2]))
        elif len(word) > 1 and word.startswith(s) and word[-2] in vs and word.endswith('м') and len(word) - len(s) == 2 and word not in exep:
          if word[-2:] == 'ам' or word[-2:] == 'ям':
            if s[-1] == 'и' or s[-1] == 'ы':
              found.append((s, i[1], i[2]))
          else:
            found.append((s, i[1], i[2]))
        elif len(word) > 1 and word.startswith(s) and word[-2] in vs and word.endswith('в') and len(word) - len(s) == 2 and word not in exep:
            if s[-1] == 'и' or s[-1] == 'ы':
              found.append((s, i[1], i[2]))
        elif len(word) > 1 and word.startswith(s) and word[-2] in vs and word.endswith('й') and len(word) - len(s) == 2 and word not in exep:
            found.append((s, i[1], i[2]))
        elif len(word) > 1 and word.startswith(s) and word[-2] in vs and word.endswith('х') and len(word) - len(s) == 2 and word not in exep:
            if s[-1] == 'и' or s[-1] == 'ы':
              found.append((s, i[1], i[2]))

  for i in found:
    if isinstance(i, str) == True:
      if i in to_append:
        found.append(to_append[i])
    else:
      if i[0] in to_append:
        found.append(to_append[i[0]])

  ffound = []
  for line in found:
    if line[0] not in opts:
      ffound.append(line)
    else:
      opt = opts[line[0]]
      f = False
      for i in opt:
        sp = i[0].split()
        for s in sp:
          if s[:-1] in line[2].lower():
            ffound.append((i[1], line[1], line[2]))
            f = True
      if f == False:
        ffound.append(line)
  return ffound

In [0]:
up_words = list(set([x for x in upper_words if x[0] not in [x[0] for x in found]]))
postfound = capital_search(up_words)
postfound = [x for x in postfound if str(x[0]).endswith('ам') == False]
ffound = ffound + postfound

Приведение рубрик к рубрикам из бумажного указателя

In [0]:
res = []
for i in ffound:
  if i[0] in repl:
    if len(i) > 1:
      res.append((repl[i[0]], i[1], i[2]))
    else:
      res.append([repl[i[0]]])
  else:
    res.append(i)

res = [x for x in res if x[0] not in rusforms]
# rres = [str(x[0]).lower() for x in res]
# rres = [x for x in rres if len(x) > 1]

Отметение вариантов лемм по окончанию

In [0]:
options = {}
clean = []

for i in res:
  if len(i) == 3:
    k = str(i[1])
    if (k, i[2]) not in options:
      options[(str(i[1]), i[2])] = [i[0]]
    else:
      options[(str(i[1]), i[2])].append(i[0])
  else:
    clean.append(i[0])

opts = {}
for k, vs in options.items():
  if len(set(vs)) > 1:
    f = False
    for v in vs:
      if re.search('[–\,\(\)\.-]', v):
        clean.append([v][0])
        f = True
        break
    if f == False:
      opts[k] = list(set(vs))
  else:
    clean.append(list(set(vs))[0])

In [0]:
# са/си, ла/ли, ра/ри, на/ни, за/зи, фа/фи, ва/ви, па/пи, да/ди, ма/ми, та/ти, ба/би.
# буквосочетания выше выделены искусственно (из всех возможных согласных на конце слова перед -и или -а)

#шардула/шардули; ракшаса/ракшаси; 

solid = ['с', 'л', 'р', 'н', 'з', 'ф', 'в', 'п', 'д', 'м', 'т', 'б']
vos = ['а', 'я', 'ю', 'е', 'о', 'ы', 'и']

solid_opts = {}
to_set = {}
for_dpavlov = {}

for k, vs in opts.items():
  for indx, i in enumerate(k[1].split()):
    if str(indx) == k[0]:
      for v in vs:
        #ракшасов
        #на -в могут заканчиваться слова только во мн. числе, поэтому лемма должна заканчиваться на -и или -ы, т.е. стоять во мн. числе
        if i[-1] == 'в':
          if len([m for m in vs if m.endswith('ы') or m.endswith('и')]) > 0:
            if v[-1] == 'ы' or v[-1] == 'и':
              if k not in solid_opts:
                solid_opts[k] = [v]
              else:
                solid_opts[k].append(v)
        elif i.endswith('ой'):
          # if len([m for m in vs if m.endswith('а')]) > 0:
          if v[-1] != 'ы' and v[-1] != 'и':
            if k not in solid_opts:
              solid_opts[k] = [v]
            else:
              solid_opts[k].append(v)
        
        #читрами
        #на -ми могут заканчиваться слова только во мн. числе, поэтому лемма должна заканчиваться на -и или -ы, т.е. стоять во мн. числе
        elif i.endswith('ми'):
          if len([m for m in vs if m.endswith('ы') or m.endswith('и')]) > 0:
            if v[-1] == 'ы' or v[-1] == 'и':
              if k not in solid_opts:
                solid_opts[k] = [v]
              else:
                solid_opts[k].append(v)

        #в тексте слово "шастрах", варианты леммы - "шастра", "шастры"
        elif i[-1] == 'х':
          if len([m for m in vs if m.endswith('ы') or m.endswith('и')]) > 0:
            #если лемма заканчивается на "ы" или "и", то выбираем её (так как слово в тексте заканчивается на "х", соответственно, стоит во мн. числе)
            #и лемма должна быть во мн. числе
            if v[-1] == 'ы' or v[-1] == 'и':
              #сохраняется лемма, предложение и индекс ее словоформы в предложении для дальнейшей обработки
              if k not in solid_opts:
                solid_opts[k] = [v]
              else:
                solid_opts[k].append(v)

        #шардулу
        #на -у могут заканчиваться слова только во ед. числе, поэтому лемма не должна заканчиваться на -и или -ы, т.е. стоять в ед. числе
        elif i[-1] in ['у', 'ю', 'е', 'о']:
          if v[-1] != 'ы' and v[-1] != 'и':
            if k not in solid_opts:
              solid_opts[k] = [v]
            else:
              solid_opts[k].append(v)
        
        #шардулам, валакхильям
        #на -ам и -ям могут заканчиваться слова только во мн. числе, поэтому лемма должна заканчиваться на -и или -ы, т.е. стоять во мн. числе
        elif i.endswith('ам') or i.endswith('ям'):
          if v[-1] == 'ы' or v[-1] == 'и':
            if k not in solid_opts:
              solid_opts[k] = [v]
            else:
              solid_opts[k].append(v)
        elif i.endswith('ом') or i.endswith('ем'):
          if v[-1] != 'ы' and v[-1] != 'и':
            if k not in solid_opts:
              solid_opts[k] = [v]
            else:
              solid_opts[k].append(v)

        # слово "кадали", варианты - "кадала", "кадали".
        # слово "кадала", варианты - "кадала", "кадали" - искусственный пример
        elif i[-1] in vos and i[-2] in solid:
          # оставляем "кадала", когда слово "кадала"
          if i[-1] != 'и' and v[-1] != 'и':
            if i[-1] != 'х' and i[-1] != 'в' and not i.endswith('ам') and not i.endswith('ям') and not i.endswith('ми') and v[-1] != 'ы':
              if k not in solid_opts:
                solid_opts[k] = [v]
              else:
                solid_opts[k].append(v)
          #оставляем "кадали", когда слово "кадали"
          elif i[-1] == 'и' and v[-1] == 'и':
            if k not in solid_opts:
              solid_opts[k] = [v]
            else:
              solid_opts[k].append(v)
        else:
          for_dpavlov[k] = vs

Запуск deeppavlov

In [0]:
!pip install git+https://github.com/andersjo/dependency_decoding
!pip install deeppavlov
!python -m deeppavlov install morpho_ru_syntagrus_pymorphy
!python -m deeppavlov install ru_syntagrus_joint_parsing

  Cloning https://github.com/andersjo/dependency_decoding to /tmp/pip-req-build-aagu5i9_
  Running command git clone -q https://github.com/andersjo/dependency_decoding /tmp/pip-req-build-aagu5i9_
  Created wheel for dependency-decoding: filename=dependency_decoding-0.0.1-cp36-cp36m-linux_x86_64.whl size=362135 sha256=d8bfc811505cada0bc3cb2fc63a75c4a35b117dfc9627444fa74f9bc3074dc72
  Stored in directory: /tmp/pip-ephem-wheel-cache-w1c26ebd/wheels/af/bd/80/9dbd1867930b7d7dbad1e9aed9d039357514b9accf1992beca
Successfully built dependency-decoding
     |████████████████████████████████| 839kB 2.8MB/s 
     |████████████████████████████████| 6.7MB 14.2MB/s 
     |████████████████████████████████| 7.3MB 40.8MB/s 
     |████████████████████████████████| 10.4MB 40.8MB/s 
     |████████████████████████████████| 860kB 35.2MB/s 
     |████████████████████████████████| 512kB 34.0MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |█

2020-05-29 05:14:44.787 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'morpho_ru_syntagrus_pymorphy' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/morpho_tagger/UD2.0/morpho_ru_syntagrus_pymorphy.json'
  Created wheel for russian-tagsets: filename=russian_tagsets-0.6-cp36-none-any.whl size=24636 sha256=b3122552fca104d4b95044124f6e0390efd7d4a932a0146b81d322dba80bc7fd
  Stored in directory: /root/.cache/pip/wheels/e8/9d/dd/4679aca4031fdb0d3ad65e165ba5343e61441ed7ad587a08e6
Successfully built russian-tagsets
     |████████████████████████████████| 110.5MB 98kB/s 
     |████████████████████████████████| 512kB 36.3MB/s 
     |████████████████████████████████| 3.8MB 34.6MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=311ff3f69d58840cd71341fe8bf63df640dd564b28b46131f1908f67295fa95a
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERR

In [0]:
from deeppavlov import build_model

joint_model = build_model("ru_syntagrus_joint_parsing", download=True)

2020-05-29 05:16:04.370 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'ru_syntagrus_joint_parsing' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/syntax/ru_syntagrus_joint_parsing.json'
2020-05-29 05:16:06.178 INFO in 'deeppavlov.core.data.utils'['utils'] at line 80: Downloading from http://files.deeppavlov.ai/deeppavlov_data/syntax_parser/syntax_ru_syntagrus_bert.tar.gz to /root/.deeppavlov/models/syntax_ru_syntagrus_bert.tar.gz
100%|██████████| 693M/693M [02:43<00:00, 4.23MB/s]
2020-05-29 05:18:49.984 INFO in 'deeppavlov.core.data.utils'['utils'] at line 242: Extracting /root/.deeppavlov/models/syntax_ru_syntagrus_bert.tar.gz archive into /root/.deeppavlov/models/syntax_ru_syntagrus
2020-05-29 05:18:59.389 INFO in 'deeppavlov.core.data.utils'['utils'] at line 80: Downloading from http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz to /root/.deeppavlov/downloads/rubert_cased_L-12_H-768_A-12_v1.tar.gz
100%|█████

2020-05-29 05:25:51.546 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/morpho_ru_syntagrus/tag.dict]











The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




Instructions for updating:
Use standard file APIs to check for files with this prefix.


2020-05-29 05:26:14.70 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/morpho_ru_syntagrus/model]



INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/morpho_ru_syntagrus/model


2020-05-29 05:26:15.898 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 52: No load path is set for UDPymorphyLemmatizer!
2020-05-29 05:26:16.343 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/syntax_ru_syntagrus/deps.dict]


Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor



2020-05-29 05:26:43.443 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/syntax_ru_syntagrus/model_joint]


INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/syntax_ru_syntagrus/model_joint


Запись в файл разбора deeppavlov-ым всего текста по предложениям

In [0]:
with open(path + 'volume_3.txt', 'r', encoding = 'utf-8') as r:
  text = r.read()

reg_space = re.compile('\s{2,}', re.DOTALL)
reg_punct = ("\s[0-9!\(\),-:;\?\[\]«»'–“„\.•…]+|[0-9!\(\),-:;\?\[\]«»'–“„\.•…]+\s|[0-9!\(\),-:;\?\[\]«»'–“„\.•…]+$|[\(\)«»'“„]+")
text = re.sub('\n', ' ', text)
text = re.sub('\s{2,}', ' ', text)
print(len(nltk.sent_tokenize(text)))

for indx, sent in enumerate(nltk.sent_tokenize(text)[3549:]):
  try:
    print(indx)
    if re.search('[а-яА-Я](-\s-[0-9]+-\s)[а-яА-Я]+', sent):
      br = re.search('[а-яА-Я](-\s-[0-9]+-\s)[а-яА-Я]+', sent).group(1)
      sent = re.sub(br, '', sent)

    text = re.sub(r'\d', '', sent)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'[A-Za-z]', '', text)
    text = re.sub(reg_punct, ' ', text)

    if re.search(reg_space, text):
      text = re.sub(reg_space, ' ', text)

    predictions = joint_model([text])
    with open(path + 'deeppavlov_mbh_3.txt', 'a', encoding = 'utf-8') as j:
      j.write('(')
      j.write(text)
      j.write(',')
      j.write(str(predictions))
      j.write(')')
      j.write('\n')
  except:
    continue

Streaming output truncated to the last 5000 lines.
8539
8540
8541
8542
8543
8544
8545
8546
8547
8548
8549
8550
8551
8552
8553
8554
8555
8556
8557
8558
8559
8560
8561
8562
8563
8564
8565
8566
8567
8568
8569
8570
8571
8572
8573
8574
8575
8576
8577
8578
8579
8580
8581
8582
8583
8584
8585
8586
8587
8588
8589
8590
8591
8592
8593
8594
8595
8596
8597
8598
8599
8600
8601
8602
8603
8604
8605
8606
8607
8608
8609
8610
8611
8612
8613
8614
8615
8616
8617
8618
8619
8620
8621
8622
8623
8624
8625
8626
8627
8628
8629
8630
8631
8632
8633
8634
8635
8636
8637
8638
8639
8640
8641
8642
8643
8644
8645
8646
8647
8648
8649
8650
8651
8652
8653
8654
8655
8656
8657
8658
8659
8660
8661
8662
8663
8664
8665
8666
8667
8668
8669
8670
8671
8672
8673
8674
8675
8676
8677
8678
8679
8680
8681
8682
8683
8684
8685
8686
8687
8688
8689
8690
8691
8692
8693
8694
8695
8696
8697
8698
8699
8700
8701
8702
8703
8704
8705
8706
8707
8708
8709
8710
8711
8712
8713
8714
8715
8716
8717
8718
8719
8720
8721
8722
8723
8724
8725
8726
8727
8728

In [0]:
# чтение файла с синт. разбором предложений текста

d = {}

with open(path + 'deeppavlov_mbh_3.txt', 'r', encoding = 'utf-8') as r:
  for i in r:
    if re.search('\((.*)\,(\[.*])', i):
      sent = re.search('\((.*)\,(\[.*])', i).group(1)
      synt = re.search('\((.*)\,(\[.*])', i).group(2)
      d[sent] = synt

Отметение оставшихся вариантов лемм с помощью разбора deeppavlov

In [0]:
#получение всех словоформ леммы в тексте

def get_wordforms(word):

  with open(path + 'Рамаяна_3.txt', 'r', encoding = 'utf-8') as r:
    text = r.read()

  reg_space = re.compile('\s{2,}', re.DOTALL)
  reg_punct = ("\s[0-9!\(\),-:;\?\[\]«»'–“„\.•…]+|[0-9!\(\),-:;\?\[\]«»'–“„\.•…]+\s|[0-9!\(\),-:;\?\[\]«»'–“„\.•…]+$|[\(\)«»'“„]+")
  if re.search('[а-яА-Я](-\s-[0-9]+-\s)[а-яА-Я]+', text):
    br = re.search('[а-яА-Я](-\s-[0-9]+-\s)[а-яА-Я]+', text).group(1)
    text = re.sub(br, '', text)
  text = re.sub(r'\d', '', text)
  text = re.sub(r'\n', ' ', text)
  text = re.sub(r'[A-Za-z]', '', text)
  text = re.sub(reg_punct, ' ', text)
  if re.search(reg_space, text):
    text = re.sub(reg_space, ' ', text)
  text = text.lower()

  forms = []
  for i in text.split():
    if i == word:
      forms.append(i)
    elif word.startswith(i[:-1]) and len(word) == len(i) and len(i) > 3:
      forms.append(i)
    elif word.startswith(i[:-2]) and len(word) - len(i) < 3 and i[-1] in ['в', 'й', 'х', 'м'] and len(i) > 3:
      forms.append(i)
    elif word.startswith(i[:-3]) and len(i) - len(word) < 3 and i.endswith('ми') and len(i) > 4:
      forms.append(i)
  
  return forms

In [0]:
%%time

clean_case = []

vs = ['а', 'я', 'ю', 'е', 'о', 'ы', 'и']

for k, v in for_dpavlov.items():

  text = k[1]
  for indx, i in enumerate(text.split()):
    if str(indx) == k[0]:
      wordform = i.lower()

  if text in d:
    elem = d[text]

    number = [i for i in range(int(k[0])-3, int(k[0])+4)]
    arr = []
    for i in number:
      if number[-1] != i:
        st = '(' + str(i) + ')|'
        arr.append(st)
      else:
        st = '(' + str(i) + ')'
        arr.append(st)

    st = ''.join(arr)
    s = '(' + st + ')'

    if re.search(r'({})\\t{}[а-яё]*[\\ta-z=|_]+case=nom[\\ta-z=|_]+\\t{}'.format(wordform, wordform[:-1], s), elem.lower()):
      if wordform in v:
        clean_case.append((wordform, k[0], k[1]))
        # print((wordform, k[0], k[1]))
        continue
      else:
        clean_case.append((wordform, k[0], k[1]))


    elif re.search(r'({})\\t{}[а-яё]*[\\ta-z=|_]+case=(gen|dat|acc|ins|loc)[\\ta-z=|_]+\\t{}'.format(wordform, wordform[:-1], s), elem.lower()):
      check = False
      for lem in v:
        #Урваши
        forms = get_wordforms(lem)
        #если слово в других формах в тексте не встречается, то принимаем его за лемму
        if bool(forms.count(forms[0]) == len(forms)) and lem == forms[0]:
          clean_case.append((lem, k[0], k[1]))
          check == True
       
      #но если слово встречается в других формах в тексте..
      #словоформа "ракшасу", варианты леммы - ракшас, ракшаси. 
      if check == False:
        if len(v) == 2:
          if v[0].endswith('и') == True and v[1].endswith('и') == False:
            clean_case.append((v[1], k[0], k[1]))
          elif v[0].endswith('и') == False and v[1].endswith('и') == True:
            clean_case.append((v[0], k[0], k[1]))
          else:
            for lem in v:
              for i in [x[0] for x in clean_case]:
                if lem[-1] in vs:
                  if i.startswith(lem[:-1]):
                    clean_case.append((i, k[0], k[1]))
                    break
    else:
      clean_case.append((wordform, k[0], k[1]))
  else:
    print('text not in d')

for x in clean_case:
  clean.append(x[0])

Объединение рубрик указателя

In [0]:
lexemes = []

for line in list(set(clean)):
  if re.match('[а-яА-Я]+', line):
    word = re.match('[а-яА-Я]+', line).group(0)
    check = False
    if word in rus_words:
      lexemes.append(line)
      continue

    wordforms = []
    wordforms.append(word)

    if word[-1] != 'ю' and word[-1] != 'у':
      for l in list(set(clean)):
        word2 = re.match('[а-яА-Я]+', l).group(0)
        if word != word2:

          if len(word) > len(word2) and len(word) - len(word2) <= 1:
            if word2.startswith(word[:-1]) or word.startswith(word2[:-1]):
              if word[-1] in vs and word2[-1] not in vs:
                if word[:-1] == word2:
                  check = True
                  wordforms.append(word2)
                  # print((word, word2))
              elif word2[-1] in vs and word[-1] not in vs:
                if word2[:-1] == word:
                  check = True
                  wordforms.append(word2)
                  # print((word, word2))
              elif word[-1] in vs and word2[-1] in vs:
                if word[:-1] == word2[:-1]:
                  check = True
                  wordforms.append(word2)
                  # print((word, word2))

          elif len(word2) > len(word) and len(word2) - len(word) <= 1:
            if word2.startswith(word[:-1]) or word.startswith(word2[:-1]):
              if word[-1] in vs and word2[-1] not in vs:
                if word[:-1] == word2:
                  check = True
                  wordforms.append(word2)
                  # print((word, word2))
              elif word2[-1] in vs and word[-1] not in vs:
                if word2[:-1] == word:
                  check = True
                  wordforms.append(word2)
                  # print((word, word2))
              elif word[-1] in vs and word2[-1] in vs:
                if word[:-1] == word2[:-1]:
                  check = True
                  wordforms.append(word2)
                  # print((word, word2))

          elif len(word2) == len(word):
            if word2.startswith(word[:-1]) or word.startswith(word2[:-1]):
              if word[-1] in vs and word2[-1] not in vs:
                if word[:-1] == word2:
                  check = True
                  wordforms.append(word2)
                  # print((word, word2))
              elif word2[-1] in vs and word[-1] not in vs:
                if word2[:-1] == word:
                  check = True
                  wordforms.append(word2)
                  # print((word, word2))
              elif word[-1] in vs and word2[-1] in vs:
                if word[:-1] == word2[:-1]:
                  check = True
                  wordforms.append(word2)
                  # print((word, word2))

    # if len(wordforms) > 1:
    if check == True:
      lexemes.append(wordforms)
    else:
      lexemes.append(line)

In [0]:
unit_clean = list(set([x for x in lexemes if isinstance(x, str)]))

united = []
for forms in lexemes:
  if [x for x in forms if x[-1] not in vs]:
    united.append([x for x in forms if x[-1] not in vs][0])
  elif [x for x in forms if x[-1] not in ['и', 'ы']]:
    united.append([x for x in forms if x[-1] not in ['и', 'ы']][0])

for i in unit_clean:
  united.append(i)

Запись получившегося автоматического указателя в файл

In [0]:
with open(path + 'automated_index.txt', 'w', encoding='utf-8') as m:
  for i in sorted(list(set(united))):
    m.write(i)
    m.write('\n')

Нахождение всех словоформ рубрик из указателя в тексте

In [0]:
united_forms = {}
for line in list(set(united)):
  if re.match('[а-яА-Я]+', line):
    word = re.match('[а-яА-Я]+', line).group(0)
    if word not in rus_index_one:
      forms = get_wordforms(word)
      forms = [x for x in forms if x not in rus_words]
      forms.append(word)
      united_forms[line] = set(forms)
    else:
      for k, v in rus_index.items():
        if word in k:
          w = v
          w.append(k)
          united_forms[line] = set(w)

In [0]:
with open(path + 'automated_index_forms.txt', 'w', encoding='utf-8') as m:
  for k, v in united_forms.items():
    m.write(k)
    m.write(' : ')
    m.write(str(v))
    m.write('\n')